In [1]:
from random import choice, randint

D = 'Defect'
C = 'Cooperate'

rewards = {(C, C): (3, 3), (C, D): (0, 5), (D, C): (5, 0), (D, D): (2, 2)}

options = [C, D]
seq = [C, C, D]

In [2]:
# O strategie de joc este caracterizată de o funcție care întoarce, la fiecare apel, acțiunea aleasă de strategie.
# Parametrul primit de funcție este o listă de tupluri care conține acțiunile jucate anterior în cursul aceluiași 
# joc iterat.
# Fiecare tuplu din listă corespunde unui joc individual de dilema prizonierului și conține pe prima poziție 
# acțiunea aleasă de acest agent și pe a doua acțiunea jucată de oponent.

def AllD(_):
    return D


def Random(_):
    return choice(options)


def TFT(information):
    return C if not information else information[len(information) - 1][1]


def Joss(information):
    tft_opt = TFT(information)
    if tft_opt == C and randint(1, 10) == 1:
        return D
    return tft_opt
    

def Tester(information):
    if not information:
        return D
    if D in map(lambda p: p[1], information):
        return TFT(information)

    last_games = list(map(lambda p: p[0], information))
    if len(last_games) < 3:
        return C
    return seq[(len(last_games) + 1) % len(seq)]


def _last_games_D(last_games, n):
    return len(last_games) >= n and C not in last_games[-n:]


def Custom(information):
    last_games = list(map(lambda p: p[0], information))
    if _last_games_D(last_games, 2):
        return D
    return TFT(information)

In [3]:
availableStrategies = [
    ('All-D', AllD),
    ('Random', Random),
    ('Tit-For-Tat', TFT),
    ('Joss', Joss),
    ('Tester', Tester),
    ('Custom', Custom)
]

In [4]:
strategies = []
for (name, proc) in availableStrategies:
    strategies.append({'name': name, 'procedure': proc, 'wins': 0, 'score': 0, 'games': 0, 'plays': {}})

# joacă un joc între A și B, întoarce recompensele asociate
def play_game(players, verbose = False):
    choices = [p['strategy']['procedure'](p['information']) for p in players]
    for i in range(2):
        players[i]['information'].append((choices[i], choices[1 - i]))
    if verbose: print(players[0]['strategy']['name']+" vs "+players[1]['strategy']['name']+" choices: "+str(choices)+" rewards: "+str(rewards[tuple(choices)]))
    return rewards[tuple(choices)]
    
# joacă `iterations` jocuri între A și B, întorcând scorul asociat întregului joc iterat
def play_iterated_pd(players, n_iterations, verbose = False):
    score = (0, 0)
    for i in range(n_iterations):
        rewardsAB = play_game(players, verbose)
        score = tuple([score[pi] + rewardsAB[pi] for pi in range(2)])
    if verbose: print("== result: "+str(score))
    return score

# joacă un turneu de n jocuri de câte n iterații, alegând aleator între strategiile date în `strategies`
def tournament(n_games, n_iterations, strategies, verbose = False):
    for game in range(n_games):
        agents = []
        strat = []
        for i in range(2):
            agents.append({'strategy': choice(strategies), 'information': []})
            strat.append(agents[i]['strategy'])
        for i in range(2):
            for j in range(2):
                if i != j:
                    if strat[j]['name'] not in strat[i]['plays']:
                        strat[i]['plays'][strat[j]['name']] = 1
                    else:
                        strat[i]['plays'][strat[j]['name']] += 1
        scores = play_iterated_pd(agents, n_iterations, verbose)
        result = (0, 0)
        if scores[0] > scores[1]:
            result = (1, 0)
        if scores[0] < scores[1]:
            result = (0, 1)
        for i in range(2):
            strat[i]['wins'] += result[i]
            strat[i]['score'] += scores[i]
            strat[i]['games'] += 1
    print('\n\n================ total games: ' + str(n_games))
    for s in strategies:
        print('\n strategy ' + s['name'])
        if s['games']:
            plays = ' played against '
            for s_op in strategies:
                if s_op['name'] in s['plays']:
                    plays += s_op['name'] + ' (' + str(s['plays'][s_op['name']]) + ') '
            print('\t' + plays)
            print( 
              '\t played '+str(s['games'])+' times and won '+str(s['wins'])+' times with a global score of '+str(s['score']) +
                  '\n\t score/games: '+str(round(float(s['score'])/s['games'], 2))+ 
                  '\t wins/games: '+str(round(float(s['wins'])/s['games'], 2))+
                  '\t score/wins: '+(str(round(float(s['score'])/s['wins'], 2)) if s['wins'] else "--"))
        else:
            print("\t played no games.")
        
        
# tournament(50, 10, strategies, True) # test, with Verbose
tournament(500, 100, strategies) # short
tournament(5000, 100, strategies) # long



================ total games: 500

 strategy All-D
	 played against All-D (22) Random (19) Tit-For-Tat (31) Joss (34) Tester (33) Custom (31) 
	 played 170 times and won 111 times with a global score of 37177
	 score/games: 218.69	 wins/games: 0.65	 score/wins: 334.93

 strategy Random
	 played against All-D (19) Random (28) Tit-For-Tat (32) Joss (25) Tester (24) Custom (20) 
	 played 148 times and won 34 times with a global score of 31011
	 score/games: 209.53	 wins/games: 0.23	 score/wins: 912.09

 strategy Tit-For-Tat
	 played against All-D (31) Random (32) Tit-For-Tat (34) Joss (33) Tester (28) Custom (29) 
	 played 187 times and won 0 times with a global score of 47129
	 score/games: 252.03	 wins/games: 0.0	 score/wins: --

 strategy Joss
	 played against All-D (34) Random (25) Tit-For-Tat (33) Joss (40) Tester (25) Custom (30) 
	 played 187 times and won 98 times with a global score of 40707
	 score/games: 217.68	 wins/games: 0.52	 score/wins: 415.38

 strategy Tester
	 played 